In [1]:
import re
import torch
import json
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset

In [4]:
quac = load_dataset("quac", split = "train")

Reusing dataset quac (/home/willy/.cache/huggingface/datasets/quac/plain_text/1.1.0/4170258e7e72d7c81bd6441b3f3489ea1544f0ff226ce61e22bb00c6e9d01fb6)


In [13]:
quac[0].keys()

dict_keys(['dialogue_id', 'wikipedia_page_title', 'background', 'section_title', 'context', 'turn_ids', 'questions', 'followups', 'yesnos', 'answers', 'orig_answers'])

In [14]:
data = quac[0]

In [16]:
data["questions"]

['Where is Malayali located?',
 'What other languages are spoken there?',
 'What else is this place known for?',
 'Were they ever successful in doing this?',
 'Do they produce anything from here?',
 'Is this population still growing?',
 'Is the country thriving?']

In [19]:
for i in range(5):
    print(data["answers"]["texts"][i])
    print(data["orig_answers"]["texts"][i])

['30,803,747 speakers of Malayalam in Kerala, making up 93.2% of the total number of Malayalam speakers in India,']
30,803,747 speakers of Malayalam in Kerala, making up 93.2% of the total number of Malayalam speakers in India,
['33,015,420 spoke the standard dialects, 19,643 spoke the Yerava dialect and 31,329 spoke non-standard regional variations like Eranadan.']
33,015,420 spoke the standard dialects, 19,643 spoke the Yerava dialect and 31,329 spoke non-standard regional variations like Eranadan.
['World Malayalee Council, the organisation working with the Malayali diaspora across the Globe has embarked upon a project for making a data bank of the diaspora.']
World Malayalee Council, the organisation working with the Malayali diaspora across the Globe has embarked upon a project for making a data bank of the diaspora.
['CANNOTANSWER']
CANNOTANSWER
['CANNOTANSWER']
CANNOTANSWER


In [20]:
data["answers"]

{'texts': [['30,803,747 speakers of Malayalam in Kerala, making up 93.2% of the total number of Malayalam speakers in India,'],
  ['33,015,420 spoke the standard dialects, 19,643 spoke the Yerava dialect and 31,329 spoke non-standard regional variations like Eranadan.'],
  ['World Malayalee Council, the organisation working with the Malayali diaspora across the Globe has embarked upon a project for making a data bank of the diaspora.'],
  ['CANNOTANSWER'],
  ['CANNOTANSWER'],
  ['In 2010, the Census of Population of Singapore reported that there were 26,348 Malayalees in Singapore.'],
  ['CANNOTANSWER']],
 'answer_starts': [[51], [640], [1862], [2024], [2024], [1461], [2024]]}

In [25]:
answers = [answer[0] for answer in data["answers"]["texts"]]

In [27]:
"CANNOTANSWER" in answers

True

In [30]:
answer_starts = [answer_start[0] for answer_start in data["answers"]["answer_starts"][:2]]


In [32]:
answers

['30,803,747 speakers of Malayalam in Kerala, making up 93.2% of the total number of Malayalam speakers in India,',
 '33,015,420 spoke the standard dialects, 19,643 spoke the Yerava dialect and 31,329 spoke non-standard regional variations like Eranadan.',
 'World Malayalee Council, the organisation working with the Malayali diaspora across the Globe has embarked upon a project for making a data bank of the diaspora.',
 'CANNOTANSWER',
 'CANNOTANSWER',
 'In 2010, the Census of Population of Singapore reported that there were 26,348 Malayalees in Singapore.',
 'CANNOTANSWER']

In [33]:
index = answers.index("CANNOTANSWER")

In [36]:
data["questions"]

['Where is Malayali located?',
 'What other languages are spoken there?',
 'What else is this place known for?',
 'Were they ever successful in doing this?',
 'Do they produce anything from here?',
 'Is this population still growing?',
 'Is the country thriving?']

In [38]:
ans_questions = [question for question in data["questions"][:2]]

In [39]:
ans_questions

['Where is Malayali located?', 'What other languages are spoken there?']

In [41]:
count = 0
for data in quac:
    answer_starts = [answer_start[0] for answer_start in data["answers"]["answer_starts"][:2]]
    print(answer_starts[-1])
    if answer_starts[-1] > 400:
        count += 1

640
51
147
89
712
0
1450
56
2853
2253
772
991
391
145
358
40
490
1739
77
4352
473
193
426
795
84
63
154
991
206
236
2365
87
144
1065
167
621
279
179
2169
97
91
501
1155
83
2606
2084
1942
5235
1770
446
76
68
370
2002
421
1920
1531
24
165
34
310
2057
797
494
116
103
968
20
815
3164
218
1542
2040
1810
0
9
1700
1042
133
2574
42
80
709
130
584
753
572
1185
1248
193
200
731
2321
279
245
87
3852
382
72
533
66
2126
0
1887
3
303
444
888
2745
70
215
127
153
30
858
377
136
1319
439
1343
87
221
191
1760
2562
503
1974
995
1841
0
792
2735
742
897
183
139
808
2127
204
661
105
1960
194
0
489
323
322
0
10
73
175
799
148
2193
2050
67
30
1348
142
571
766
70
325
84
0
1868
452
216
132
565
680
19
111
702
173
1052
158
38
2343
70
1305
211
2965
398
171
151
150
161
2164
30
1466
0
19
118
578
1916
0
174
503
5448
179
1281
842
1812
2067
1392
1307
68
100
175
387
44
252
1728
541
813
72
748
2435
2698
659
1017
1699
537
1494
244
2112
207
35
246
96
232
371
140
514
132
136
561
115
256
1984
841
264
0
768
213
723
62
1307
18

In [42]:
count

5919